# Design of map plot
If we plot maps, we can actually decide quite a lot in how they should look like. But first, we have to import all necessary packages.

In [ ]:
import osmnx as ox
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import contextily
from IPython.display import Image

%matplotlib inline
ox.config(log_console=True)
ox.__version__

## Change colour of OSMnx plots
Well, let us start, with what we already know. OSMnx offers some possibilities to change colours by using **bgcolor** and **Edge_color**. So we can simply change the colour of our plot.

In [ ]:
#Setting values for parameters
dpi=72
size=600
place = "Aachen, Germany"
fp = f"./aachen_col.png"
#Querying OSM data using a city name
fig, ax = ox.plot_figure_ground(
    address=place,
    network_type="walk",
    filepath=fp,
    dpi=dpi,
    bgcolor = "white",
    edge_color = "lightblue",
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)

In [ ]:
#Defining required tags for quering OSM
tags={"building":True}
point = (50.775593, 6.082478)
size=500
dist = 1000
#OSMnx method geometries_from_point queryies OSM data using a point
gdf = ox.geometries_from_point(point, tags, dist=dist)
gdf_proj = ox.project_gdf(gdf)

In [ ]:
#Quering OSM by using the defined point
bbox = ox.utils_geo.bbox_from_point(point=point, dist=dist, project_utm=True)
fp = f"./aachen_col_plan.png"
#Figure ground plan change colour
fig, ax = ox.plot_footprints(
    gdf_proj,
    bbox=bbox,
    bgcolor="white",
    color="lightblue",
    filepath=fp,
    dpi=300,
    save=True,
    show=False,
    close=True,
)
Image(fp, height=size, width=size)

## Query graph and colour them
Let us query OSM for information around the centre point of Aachen. But we can go ahead, as the **.plot_graph** method offers a possibility of changing edge, and node colours, as well as node sizes.

In [ ]:
point = (50.933594, 6.961899)
size=500
dist = 1000
tags={"building":True}
#OSMnx method geometries_from_point queryies OSM data using a point
gdf = ox.graph_from_point(point, dist=dist, network_type='all')
type(gdf)

We set colours for the edges and our background, and we just plot the graph. Try to change the node size from 10 to 1, you will see, they visually disappear. You can further change the background colour.

In [ ]:
backgr = "#061529"
edges = "#ffffff"
fig, ax = ox.plot_graph(gdf, bgcolor=backgr, node_color=edges, node_size=10, edge_color=edges)

But we can even go a little further by unpacking what is in *edge* and separately save it to variables.

In [ ]:
u_new = []
v_new = []
key_new = []
data_new = []
for u, v, keys, weight in gdf.edges(keys=True, data=True):
    u_new.append(u)
    v_new.append(v)
    key_new.append(keys)
    data_new.append(weight)

Let us assign a colour in dependency of how big a street is.

In [ ]:
roadColors = []
roadWidths = []

#Seperating streets by length
for item in data_new:
    if "length" in item.keys():
        if item["length"] <= 100:
            linewidth = 1.0
            color = "#a6a6a6" 
            
        elif item["length"] > 100 and item["length"] <= 200:
            linewidth = 1.5
            color = "#676767"
            
        elif item["length"] > 200 and item["length"] <= 400:
            linewidth = 2.0
            color = "#454545"
            
        elif item["length"] > 400 and item["length"] <= 800:
            color = "#bdbdbd"
            linewidth = 2.5
        else:
            color = "#d5d5d5"
            linewidth = 3.0

        if "primary" in item["highway"]:
            linewidth = 3.5
            color = "#ffff"
    else:
        color = "#a6a6a6"
        linewidth = 1.0
            
    roadColors.append(color)
    roadWidths.append(linewidth)

In [ ]:
fig, ax = ox.plot_graph(gdf, bgcolor=backgr, node_color=edges, node_size=0, edge_color=roadColors, 
                        edge_linewidth=roadWidths, edge_alpha=1)

## Query geometries and colour them
Now, we are ready to query geometries from OSM, here buildings around a specific point in Aachen.

In [ ]:
area = ox.geocode_to_gdf("Aachen, Germany")
gdf_new = ox.geometries_from_point(point, tags, dist=dist)
type(gdf_new)

In [ ]:
gdf_new

The following two code cells are all about changing some colouring. You probably already recognized, that the colour definition in given in *hexadecimal*.

In [ ]:
fix, ax = plt.subplots(figsize=(15,15))
gdf_new.plot(ax=ax, edgecolor='#1B215F', linewidth=1.5, facecolor="#ffffff")
plt.show()

In [ ]:
fix, ax = plt.subplots(figsize=(15,15))
gdf_new.plot(ax=ax, edgecolor='#1B215F', linewidth=1.5, facecolor="#FF0083")
plt.show()

## Adding graph and geometries in one plot
Next, we add the edges, which we get using the **.graph_from_point** method further up. We already coloured them in different grey shades. So, next we add them to our coloured plot.

In [ ]:
nodes, edges = ox.graph_to_gdfs(gdf)

In [ ]:
fix, ax = plt.subplots(figsize=(15,15))
edges.plot(ax=ax, linewidth=roadWidths, edgecolor=roadColors)
gdf_new.plot(ax=ax, edgecolor='#000000', linewidth=0.5, facecolor="#5A94CE")
plt.show()

# Using Python package ipyleaflet creating interactive maps
Ipyleaflet is an interactive widgets' library, based on ipywidgets. Everything in ipyleaflet is interactive, attributes can be updated dynamically.

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
from ipyleaflet import Marker, Icon, Map
from ipyleaflet import Map, Marker, Popup
from ipywidgets import HTML
import osmnx as ox
from IPython.display import Image

%matplotlib inline
ox.config(log_console=True)
ox.__version__

Let us create our first interactive plane map. Therefore, we need a centre point, a zoom level. You could actually add a specific [basemap](https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html).

In [ ]:
center = (50.775593, 6.082478)
map_ac = Map(center=center, zoom=15)
map_ac

In [ ]:
#change zoom level and another basemap
map_ac_new = Map(center=center, zoom = 3, basemap= basemaps.Esri.WorldTopoMap)
map_ac_new

Aachen is still the centre point, but due to zoom level, one can not see it any more.
## Creating icons 
We can do more than just create a map with a centre. So let us set some interactive icons. The following icon is the official leaflet icon in red. Let us make sure it becomes part of the map. 
![leaf](./img/leaf-red.png)

In [ ]:
center = (50.775593, 6.082478)
ac_m_ic = Map(center=center, zoom=10)
icon = Icon(icon_url='https://leafletjs.com/examples/custom-icons/leaf-red.png', icon_size=[38, 95], icon_anchor=[22,94])
mark = Marker(location=center, icon=icon, rotation_angle=90, rotation_origin='22px 94px')
ac_m_ic.add_layer(mark);
ac_m_ic

That was an easy one, because it is the official example for dynamic icons on maps. So, let us use our own icon. Anyway, it is interactive, so try to grab the tree and move it around.  
<div>
<img src="./img/baum.png" width="100"/>
</div>

In [ ]:
center = (50.775593, 6.082478)
ac_1m = Map(center=center, zoom=14)
icon = Icon(icon_url='../../files/img/baum.png', icon_size=[70, 70], icon_anchor=[22,94])
mark = Marker(location=center, icon=icon)
ac_1m.add_layer(mark);
ac_1m

## Setting Marker for location
Probably everyone knows the location icon. In Google Maps it is usually red, here we use a blue one.

In [ ]:
center = (50.775593, 6.082478)
ac_c = Map(center=center, zoom=15)
marker = Marker(location=center, draggable=False)
ac_c.add_layer(marker);
ac_c

In [ ]:
message = HTML()
message.value = "A message for you!"
popup = Popup(
    location=center,
    close_button=False,
    auto_close=False,
    close_on_escape_key=False
)
ac_c.add_layer(popup)
marker.popup = message
ac_c

We just marked the centre point of Aachen. But there are maybe even more places to be marked in one map. Easy, first, we need to get the coordinates of our *other places*, in this example let us use the *Reiff Museum Aachen* and *Aachen Hauptbahnhof (central station)*. We query their coordinates using OSMnx and add them in a list. Next, we set a marker for every element in that list.

In [ ]:
reiff = ox.geocode("reiff museum aachen")
hbf = ox.geocode("Aachen Hauptbahnhof")
l = [center, reiff, hbf]

In [ ]:
ac_c = Map(center=center, zoom=14)
for elem in l:
    marker = Marker(location=elem, draggable=False)
    ac_c.add_layer(marker);

ac_c

## GeoDataFrame in ipyleaflet
Well, we can not just add icons to an interactive map, but also DataFrames and GeoDataFrames. Let us use the *Reiff museum*, we already have its coordinates, that will be our centre point. Next we need a GeoDataFrame (or a DataFrame, but in this example it is a GeoDataFrame). To get one we go ahead and query OSM (using OSMnx) for geometries around the Reiff museum which are declared to be buildings within a distance of 100. Finally, we add the geometries to our map as a new layer.

In [ ]:
#Get geometries quering OSM
rdf = ox.geometries_from_point(reiff, tags={"building":True},dist=100)

In [ ]:
#Set up map, careful with the zoom level!
m = Map(center=reiff, zoom = 16)

In [ ]:
#Adding the GeoDataFrame to the map
geo_data = GeoData(geo_dataframe= rdf, style={'color': 'black', 'fillColor': '#3c758c', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6})
m.add_layer(geo_data)
m